In [2]:
% matplotlib notebook
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
import numpy as np
from IPython.display import Image


# Add modules path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from model.Translator import Translator
from model.DecisionTree import DecisionTree
from model.Rule import Rule
import preprocessor as pp


# Add modules path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Define language and translator functions
language = "es"
translator = Translator(language)
to_lang = translator.translate_to_language
to_col = translator.translate_to_column

# Load data
raw_data = pd.read_csv("sample.txt", header=0, skiprows=1, delimiter="\t", index_col=0, usecols=list(range(0, 9)),
                       parse_dates=to_lang(["Datetime"]), decimal=",",
                       date_parser=lambda x: pd.to_datetime(x, format="%Y/%m/%d %H:%M"))
# Translate column names
raw_data.columns = (to_col(raw_data.columns))

# Divide in blocks, extend dataset and clean data
block_data = pp.define_blocks(raw_data)
cleaned_block_data = pp.clean_processed_data(block_data)
extended_data = pp.extend_data(cleaned_block_data)
cleaned_extended_data = pp.clean_extended_data(extended_data)

# Create decision trees
[data, labels] = pp.prepare_to_decision_trees (cleaned_extended_data)
hyper_dt = DecisionTree(data, labels["Hyperglycemia_Diagnosis_Next_Block"])
hypo_dt = DecisionTree(data, labels["Hypoglycemia_Diagnosis_Next_Block"])
Image(hyper_dt.graph.create_png())
tree = hyper_dt.tree

In [3]:
def get_decisions (tree, children_left, children_right, decisions, stack, node_id, are_leaves):

    feature = tree.feature
    threshold = tree.threshold
    impurity = tree.impurity
    samples = tree.n_node_samples
    value = tree.value
    
    left_node = children_left[node_id]
    right_node = children_right[node_id]
    decisions.insert(node_id,stack)
    value_neg = value[node_id][0][0]
    value_pos = value[node_id][0][1]
    class_str = 'Negative' if value_neg > value_pos else 'Positive'
    if left_node != right_node:
        stack_left = stack + [
            {'Feature': int(feature[node_id]), 'Operator': '<=', 'Threshold': threshold[node_id], 
             'Impurity': impurity[node_id], 'Samples': samples[node_id], 'Number_Neg': value[node_id][0][0],
             'Number_Pos': value[node_id][0][1], 'Class': class_str}
        ]
        stack_right = stack + [
            {'Feature': int(feature[node_id]), 'Operator': '>', 'Threshold': threshold[node_id], 
             'Impurity': impurity[node_id], 'Samples': samples[node_id], 'Number_Neg': value[node_id][0][0],
             'Number_Pos': value[node_id][0][1], 'Class': class_str}
        ]
        get_decisions (tree, children_left, children_right, decisions, stack_left, left_node, are_leaves)
        get_decisions (tree, children_left, children_right, decisions, stack_right, right_node, are_leaves)
    else:
        are_leaves[node_id] = True
        stack += [
            {'Impurity': impurity[node_id], 'Samples': samples[node_id], 'Number_Neg': value_neg,
             'Number_Pos': value_pos, 'Class': class_str}
        ]
        decisions.insert(node_id, stack)

In [4]:
def getPatterns(tree):
    
    n_nodes = tree.node_count
    children_left = tree.children_left
    children_right = tree.children_right

    are_leaves = np.zeros(shape=n_nodes, dtype=bool)
    stack = [] 
    decisions = []
    
    get_decisions (tree, children_left, children_right, decisions, stack, 0, are_leaves)
    for id_node, node in enumerate(decisions):
        for idx_decision, decision in enumerate(decisions[id_node]):
            if 'Feature' in decisions[id_node][idx_decision]:
                feature_idx = decisions[id_node][idx_decision]['Feature']
                if isinstance(feature_idx, int):
                    value = data.columns.values[feature_idx]
                    decisions[id_node][idx_decision]['Feature'] = value
    leaves = []
    for idx, leaf in enumerate(are_leaves):
        if leaf:
            leaves.append(decisions[idx])
    patterns = []
    # Last element contains the leaf information
    valid_leaves = [leaf for leaf in leaves if leaf[-1]['Class'] == 'Positive']
    for leaf in valid_leaves:
        patterns.append(
            {"rules": [Rule(node["Feature"], node["Operator"], node["Threshold"], translator) for node in leaf[:-1]],
             "samples": leaf[-1]["Samples"],
             "impurity": leaf[-1]["Impurity"],
             "number_pos": leaf[-1]["Number_Pos"],
             'number_neg': leaf[-1]["Number_Neg"]
            })  
    return patterns

In [5]:
tree = hyper_dt.tree
patterns = getPatterns(tree)
patterns

[{'impurity': 0.0,
  'number_neg': 0.0,
  'number_pos': 210.0,
  'rules': [<model.Rule.Rule at 0x7f3048636780>,
  'samples': 210},
 {'impurity': 0.20244897959183672,
  'number_neg': 24.0,
  'number_pos': 186.0,
  'rules': [<model.Rule.Rule at 0x7f3048636898>],
  'samples': 210}]

In [6]:
for rule in patterns[0]["rules"]:
    print(rule)

Nivel de glucosa máximo del bloque anterior es menor o igual que 234.5000
Media del nivel de glucosa del bloque anterior es menor o igual que 145.3208
Nivel de glucosa máximo del día anterior es mayor que 192.0000
Desviación estandar del nivel de glucosa del bloque anterior es menor o igual que 40.9904
Desviación estandar del nivel de glucosa del día anterior es menor o igual que 40.9517
